# Main

Projeto de técnicas de programação

- Guilherme Araújo Mendes de Souza

Bibliotecas

In [208]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

Leitura dos dados

In [212]:
df_amazon = pd.read_csv('../Data/amazon.csv', encoding='ISO-8859-1')
df_data_p1 = pd.read_csv('../Data/output_data_part1.csv', sep= ';', index_col= False)
df_data_p2 = pd.read_csv('../Data/output_data_part2.csv', sep= ';', index_col= False)

C:\Users\guilh\AppData\Local\Temp\ipykernel_17264\1351230702.py:2: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df_data_p1 = pd.read_csv('../Data/output_data_part1.csv', sep= ';', index_col= False)
C:\Users\guilh\AppData\Local\Temp\ipykernel_17264\1351230702.py:3: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df_data_p2 = pd.read_csv('../Data/output_data_part2.csv', sep= ';', index_col= False)


Pré-vizualização dos dados

In [213]:
df_amazon.head()

,year,state,month,number,date
0,1998,Acre,Janeiro,0.0,1998-01-01
1,1999,Acre,Janeiro,0.0,1999-01-01
2,2000,Acre,Janeiro,0.0,2000-01-01
3,2001,Acre,Janeiro,0.0,2001-01-01
4,2002,Acre,Janeiro,0.0,2002-01-01


In [214]:
df_data_p1.head()

,CodigoOMM,NomeEstacao,Estado,EstadoDesc,EstadoRegiao,Latitude,Longitude,Altitude,EstacaoStituacao,OperanteDesde,...,UmidadeRelativa,PressaoAtmEstacao,PressaoAtmMar,DirecaoVento,VelocidadeVentoInsolacao,Nebulosidade,Evaporacao Piche,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media
0,82989,AGUA BRANCA,AL,Alagoas,Nosdeste,-9.28,-37.9,605.34,Estação Operante,05/05/1928,...,91.0,943.6,1010.2,14.0,7.0,11.3,0.0,7.8,25.92,89.0
1,82989,AGUA BRANCA,AL,Alagoas,Nosdeste,-9.28,-37.9,605.34,Estação Operante,05/05/1928,...,88.0,945.7,1012.3,5.0,5.0,NaN,1.0,NaN,NaN,NaN
2,82989,AGUA BRANCA,AL,Alagoas,Nosdeste,-9.28,-37.9,605.34,Estação Operante,05/05/1928,...,89.0,943.7,1010.1,14.0,5.0,10.5,0.0,8.4,27.26,87.0
3,82989,AGUA BRANCA,AL,Alagoas,Nosdeste,-9.28,-37.9,605.34,Estação Operante,05/05/1928,...,82.0,944.5,1010.8,5.0,7.0,NaN,3.0,NaN,NaN,NaN
4,82989,AGUA BRANCA,AL,Alagoas,Nosdeste,-9.28,-37.9,605.34,Estação Operante,05/05/1928,...,90.0,942.7,1008.7,14.0,5.0,8.9,0.0,10.5,26.68,87.0


In [215]:
df_data_p2.head()

,CodigoOMM,NomeEstacao,Estado,EstadoDesc,EstadoRegiao,Latitude,Longitude,Altitude,EstacaoStituacao,OperanteDesde,...,UmidadeRelativa,PressaoAtmEstacao,PressaoAtmMar,DirecaoVento,VelocidadeVentoInsolacao,Nebulosidade,Evaporacao Piche,Temp Comp Media,Umidade Relativa Media,Velocidade do Vento Media
0,82791,PATOS,PB,Paraiba,Nordeste,-7.01,-37.26,249.09,Estação Operante,17/10/1975,...,53.0,981.5,1008.9,9.0,4.8,11.3,0.0,13.0,30.24,51.25
1,82791,PATOS,PB,Paraiba,Nordeste,-7.01,-37.26,249.09,Estação Operante,17/10/1975,...,56.0,985.3,1012.9,9.0,4.5,NaN,7.0,NaN,NaN,NaN
2,82791,PATOS,PB,Paraiba,Nordeste,-7.01,-37.26,249.09,Estação Operante,17/10/1975,...,33.0,980.2,1007.6,14.0,3.3,NaN,6.0,NaN,NaN,NaN
3,82791,PATOS,PB,Paraiba,Nordeste,-7.01,-37.26,249.09,Estação Operante,17/10/1975,...,58.0,982.3,1008.3,5.0,4.8,10.5,9.0,9.1,29.34,56.75
4,82791,PATOS,PB,Paraiba,Nordeste,-7.01,-37.26,249.09,Estação Operante,17/10/1975,...,60.0,985.0,1012.6,5.0,3.0,NaN,7.0,NaN,NaN,NaN


Padronização das datas

In [216]:
df_data_p1['Data'] = pd.to_datetime(df_data_p2['Data'], format='%d/%m/%Y', errors='coerce')

df_data_p2['Data'] = pd.to_datetime(df_data_p2['Data'], format='%d/%m/%Y', errors='coerce')

In [217]:
df_amazon['date'] = pd.to_datetime(df_amazon['date'])
df_amazon['date'] = pd.to_datetime(df_amazon['date'], format='%d/%m/%Y')

In [218]:
df_data_p1['Data'] = pd.to_datetime(df_data_p1['Data'])
df_data_p1['Data'] = pd.to_datetime(df_data_p1['Data'], format='%d/%m/%Y')

In [219]:
df_data_p2['Data'] = pd.to_datetime(df_data_p2['Data'])
df_data_p2['Data'] = pd.to_datetime(df_data_p2['Data'], format='%d/%m/%Y')

Filtragem dos DF´s

In [220]:

df_amazon = df_amazon[(df_amazon['date'] <= pd.Timestamp(year=2017, month=12, day=31)) & (df_amazon['date'] >= pd.Timestamp(year=2012, month=1, day=1))]

df_data_p1 = df_data_p1[(df_data_p1['Data'] <= pd.Timestamp(year=2017, month=12, day=31)) & (df_data_p1['Data'] >= pd.Timestamp(year=2012, month=1, day=1))]

df_data_p2 = df_data_p2[(df_data_p2['Data'] <= pd.Timestamp(year=2017, month=12, day=31)) & (df_data_p2['Data'] >= pd.Timestamp(year=2012, month=1, day=1))]

In [221]:
df_amazon.to_csv('../Data/amazon1.csv', index=False)

df_data_p1.to_csv('../Data/output_data_part1-1.csv', index=False)

df_data_p2.to_csv('../Data/output_data_part2-1.csv', index=False)

Concatenação dos dataframe df_data_p1 e df_data_p2

In [222]:
df_data = pd.concat([df_data_p1, df_data_p2], ignore_index=True)

In [223]:
filtro = df_data[['Estado', 'EstadoDesc', 'Data', 'Precipitacao', 'TempMaxima', 'TempMinima', 'UmidadeRelativa']]

In [224]:
media_mensal = filtro.groupby(['Estado', 'EstadoDesc', pd.Grouper(key='Data', freq='M')]).mean()

media_mensal = media_mensal.reset_index()

Concatenação dos dataframe media_mensal e df_amazon

In [225]:
df_amazon.rename(columns={'state': 'EstadoDesc', 'date': 'Data'}, inplace=True)

In [226]:
df_amazon['Data'] = pd.to_datetime(df_amazon['Data'])
media_mensal['Data'] = pd.to_datetime(media_mensal['Data'])

In [227]:
df_amazon['AnoMes'] = df_amazon['Data'].dt.to_period('M')

media_mensal['AnoMes'] = media_mensal['Data'].dt.to_period('M')

In [228]:
df = pd.merge(df_amazon, media_mensal, on=['EstadoDesc', 'AnoMes'])

In [229]:
print(media_mensal)

     Estado EstadoDesc       Data  Precipitacao  TempMaxima  TempMinima  \
0        AC       Acre 2012-01-31     12.301205   30.414286   22.554217   
1        AC       Acre 2012-02-29     11.651724   30.457471   22.242529   
2        AC       Acre 2012-03-31      8.179570   31.303226   22.084946   
3        AC       Acre 2012-04-30      5.790000   31.466667   22.300000   
4        AC       Acre 2012-05-31      3.152688   31.403226   20.715054   
...     ...        ...        ...           ...         ...         ...   
1758     TO  Tocantins 2017-08-31      0.187500   35.943860   19.894690   
1759     TO  Tocantins 2017-09-30      0.000000   36.556436   21.021569   
1760     TO  Tocantins 2017-10-31      0.966897   37.564912   22.774561   
1761     TO  Tocantins 2017-11-30      5.937143   33.646364   22.498182   
1762     TO  Tocantins 2017-12-31      8.850806   31.847312   22.556989   

      UmidadeRelativa   AnoMes  
0           88.350598  2012-01  
1           87.237548  2012-02  


In [230]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 924 entries, 0 to 923
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   year             924 non-null    int64         
 1   EstadoDesc       924 non-null    object        
 2   month            924 non-null    object        
 3   number           924 non-null    float64       
 4   Data_x           924 non-null    datetime64[ns]
 5   AnoMes           924 non-null    period[M]     
 6   Estado           924 non-null    object        
 7   Data_y           924 non-null    datetime64[ns]
 8   Precipitacao     924 non-null    float64       
 9   TempMaxima       924 non-null    float64       
 10  TempMinima       924 non-null    float64       
 11  UmidadeRelativa  924 non-null    float64       
dtypes: datetime64[ns](2), float64(5), int64(1), object(3), period[M](1)
memory usage: 86.8+ KB


In [231]:
df.isnull().sum()

year               0
EstadoDesc         0
month              0
number             0
Data_x             0
AnoMes             0
Estado             0
Data_y             0
Precipitacao       0
TempMaxima         0
TempMinima         0
UmidadeRelativa    0
dtype: int64

In [232]:
df.drop(['year', 'month', 'Data_x', 'Data_y'], axis=1, inplace=True)

In [233]:
print(df)

    EstadoDesc   number   AnoMes Estado  Precipitacao  TempMaxima  TempMinima  \
0         Acre    0.000  2012-01     AC     12.301205   30.414286   22.554217   
1         Acre    0.000  2012-01     AC     12.301205   30.414286   22.554217   
2         Acre    1.000  2012-01     AC     12.301205   30.414286   22.554217   
3         Acre    1.000  2012-01     AC     12.301205   30.414286   22.554217   
4         Acre    3.000  2012-01     AC     12.301205   30.414286   22.554217   
..         ...      ...      ...    ...           ...         ...         ...   
919  Tocantins    2.397  2017-01     TO      6.517931   32.760690   22.105556   
920  Tocantins    3.536  2017-01     TO      6.517931   32.760690   22.105556   
921  Tocantins   10.737  2017-01     TO      6.517931   32.760690   22.105556   
922  Tocantins    2.969  2017-01     TO      6.517931   32.760690   22.105556   
923  Tocantins  434.000  2017-01     TO      6.517931   32.760690   22.105556   

     UmidadeRelativa  
0   

In [234]:
df.to_csv('../Data/Dataset.csv', index=False)
